<a href="https://colab.research.google.com/github/IndraLukasTjahaja/Python201_Modeling/blob/master/Credit_Risk_In_Banking_Fintech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Persiapan Google Colaboratory dan Downloading dataset

Karena data tersedia di website tjahaja, maka kita dapat langsung run dari Google Colab

# Bab III: Metodologi Data Science CRISP-DM dengan Score Card 

In [14]:
## III.2.1 Persiapan data dan impor ke Python

# library panda untuk menggunakan data frame
import pandas as pd

# Pengaturan folder tempat data / Google Colab

# Impor data dari excel xls ke Python
df = pd.read_excel('https://github.com/IndraLukasTjahaja/Digital_Credit_Risk/blob/main/Dataset/Credit_Scoring_Dataset.xlsx')
print("read datafile")
# Alternative 
# Untuk grup 1: https://github.com/IndraLukasTjahaja/Python201_Modeling/blob/master/Dataset/german_credit_easy.xlsx
# Untuk grup 2: https://github.com/IndraLukasTjahaja/Python201_Modeling/blob/master/Dataset/OnClass_CreditScoring_data_v2.xlsx  
# Untuk grup 3: https://github.com/IndraLukasTjahaja/Python201_Modeling/blob/master/Dataset/simulation_credit_kaggle_laotse.xlsx 
# Untuk yang mencari tantangan: https://github.com/IndraLukasTjahaja/Python201_Modeling/blob/master/Dataset/Fraud_Scoring_Dataset.xlsx

XLRDError: ignored

In [2]:
# Membaca file dan menera[kan tipe variabel
# Koding ini harus diganti, menyesuaikan dengan tipe data yang digunakan
df.head()

AttributeError: ignored

In [ ]:
## III.2.2 Penjelasan untuk setiap variabel di dataset

df.dtypes

In [ ]:
df.info()

In [ ]:
## III.2.4 Eksplorasi data untuk memeriksa hasil hipotesis

# Untuk melihat frekuensi dari variabel default

print(df['default'].value_counts())


In [ ]:
# Untuk melihat frekuensi dari variabel credit_history
print(df['credit_history'].value_counts())

In [ ]:
# Untuk melihat proporsi dari variabel credit_history
print(df['credit_history'].value_counts(normalize=True) * 100)

In [ ]:
# Untuk melihat proporsi dari variabel credit_history
# atau
print(df['credit_history'].value_counts() / len(df['credit_history']) * 100)

In [ ]:
# Melihat kaitan antara variabel credit_history dengan default (gagal bayar)
print(pd.crosstab(df['credit_history'], df['default']))

In [ ]:
# Melihat proporsi kaitan antara variabel credit_history dengan default (gagal bayar)
print(pd.crosstab(df['credit_history'], df['default'], normalize='index') * 100)

In [ ]:
# Rangkuman statistik untuk variabel numerik
print(df['age'].describe())

In [ ]:
# Rangkuman statistik untuk variabel numerik Age, berdasarkan default (gagal bayar atau tidak)
df.groupby('default')['age'].describe()

In [ ]:
# Menghitung korelasi antara variabel Age dengan default
print(df['default'].corr(df['age']))

In [ ]:
# Uji coba hipotesis apakah ada perbedaan nilai rata-rata variabel Age dengan default (gagal bayar dan tidak gagal bayar)
from scipy import stats
stats.ttest_ind( df[df.default == 0].age,
                 df[df.default == 1].age, nan_policy='omit')

In [ ]:
## III.2.5.1 Distribusi data untuk variabel tipe karakter

# Fungsi value_counts().plot(kind='bar')
df['credit_history'].value_counts().plot(kind='bar') 

In [ ]:
# Karena nama dari isi variabel credit_history panjang, mari kita tampilkan bar secara horizontal
df['credit_history'].value_counts().plot(kind='barh') 

In [ ]:

# atau
# Dengan menggunakan fungsi yang sama, menampilkan proporsi
# df['credit_history'].value_counts(normalize=True).plot(kind='bar') 


In [ ]:
# Nilai variabel credit_history dan proporsi default secara plot bar
pd.crosstab(df['credit_history'], df['default']).plot(kind='barh', stacked=True)

In [ ]:
# Nilai variabel credit_history dan proporsi default secara plot bar, secara proporsi
pd.crosstab(df['credit_history'], df['default'], normalize='index').plot(kind='barh', stacked=True)

In [ ]:
## III.2.5.2 Distribusi data untuk variabel tipe numerik

# Fungsi distplot() dari library seaborn
# Perhatikan bahwa distribusi plot harus tidak memiliki data kosong / missing value.
# Oleh karena itu harus menambahkan fungsi dropna() untuk secara sementara tidak
# mengikutsertakan missing value dalam pembuatan plot distribusi 

import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(10,10))
sns.distplot(df['age'].dropna(), ax=ax).set_title('Distribution Plot') 

In [ ]:
df.boxplot(column=['age'])

In [ ]:
# Menambahkan by di fungsi boxplot
df.boxplot(by='default',column=['age'])

In [ ]:
## III.2.6 Korelasi Antar Variabel

import seaborn as sns
corr = df.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)

In [ ]:
## III.3.1 Penanganan untuk data yang hilang (missing values)

# Membuat kopi dari df dan menyimpan sebagai df_prepare
df_prepare = df.copy()

In [ ]:
df.info()

In [ ]:
df_prepare.info()

In [ ]:
# Koding berikut menggantikan missing value dari semua variabel dengan
# nilai yang paling sering muncul (modus)

for column in df_prepare.columns:
        df_prepare[column].fillna(df_prepare[column].mode()[0], inplace=True)

In [ ]:
## III.3.3 Pembuatan variabel yang baru (Feature Engineering) 

df_prepare['cicilan_per_bulan'] = df_prepare['credit_amount'] / df_prepare['duration_in_month']

In [ ]:
# Fungsi boxplot dari variabel cicilan_per_bulan, dengan kemungkinan gagal bayar
df_prepare.boxplot(by='default',column=['cicilan_per_bulan'])

In [ ]:
# Menghitung korelasi dan uji coba hipotesis antara variabel cicilan_per_bulan dengan default

print(df_prepare['default'].corr(df_prepare['cicilan_per_bulan']))

stats.ttest_ind( df_prepare[df_prepare.default == 0].cicilan_per_bulan,
                 df_prepare[df_prepare.default == 1].cicilan_per_bulan, nan_policy='omit')

In [ ]:
## III.3.4 Transformasi variabel kategori menjadi numerik (One-Hot Encoding) 

# Koding untuk one hot encoding untuk semua variabel kategori

# Langkah pertama adalah membuang variabel ID karena ini adalah identifikasi nasabah
df_prepare.drop(['ID'], axis = 1, inplace = True)

# Memisahkan variabel kategori dengan variabel numerik
dataset_dummies = df_prepare.select_dtypes(exclude = ['float64','int64','int'])
dataset_int = df_prepare.select_dtypes(include=['float64','int64','int'])

# Memisahkan target/dependent variabel dari dataset dengan variabel kategori
target = df_prepare['default']
dataset_int.drop(['default'], axis = 1, inplace = True)

# Menggunakan function get_dummies dari panda untuk melakukan one hot encoding
dataset_dummies_df = pd.get_dummies(dataset_dummies,drop_first = True)
    
# Menggabungkan dataset variabel kategori dari one hot encoding dengan variabel numerik
dataset = pd.concat([dataset_dummies_df, dataset_int], axis = 1)
feature_name = dataset.columns
df_encoded = pd.concat([dataset, target], axis = 1)

In [ ]:
df_encoded.info()
df_encoded.head()

In [ ]:
# Menentukan variabel dependent (default) dan independent (input untuk model)

# X sebagai semua variabel independent, input untuk model
X = df_encoded.iloc[:,0:len(df_encoded.columns)-1]

# y sebagai target atau variabel dependent
y = df_encoded.iloc[:,len(df_encoded.columns)-1]

In [ ]:
X.head()
y.head()
print(y.value_counts())

In [ ]:
# menggunakan function SMOTE untuk melakukan kombinasi oversampling/undersampling
from imblearn.over_sampling import SMOTE

# function SMOTE dengan menerapkan proporsi 50%, maka ratio harus menjadi 1.0 Karena ratio mengacu pada proporsi gagal bayar
sm = SMOTE(random_state = 42, ratio = 1.0)

# Hasil dari function SMOTE disimpan dalam variabel X dan y
X_imbal,y_imbal = sm.fit_sample(X,y.ravel())

In [ ]:
# Melihat jumlah bayar dan gagal bayar nasabah di y_imbal hasil dari SMOTE
import collections, numpy
collections.Counter(y_imbal)

In [ ]:
# Menggunakan library dan fungsi train_test_split
from sklearn.model_selection import train_test_split

# Membagi data set menjadi train dan test, dimana proporsi test adalah 0.3 / 30% dari total data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

# Membagi lagi test data set menjadi dua, dimana 15% adalah test dan 15% digunakan sebagai validasi
X_test, X_test_holdup, y_test, y_test_holdup = train_test_split(X_test, y_test, test_size =0.5, random_state=42)

# Karena X_train dalam bentuk array, tidak akan tersimpan nama variabel independen. Oleh karena itu kita akan simpan dalam bentuk list nama-nama kolom
# Akan banyak nama kolomnya
kolom_short = ["act_0DM","act_200DM/salary","act_none","cre_his_critical/other_existing","cre_his_delay","cre_his_paid","cre_his_none/paid","purpose_business","purpose_car(new)","purpose_car(used)"    ,"purpose_appli","purpose_education","purpose_furniture","purpose_radio/television","purpose_repairs","purpose_retraining","sav_100DM","sav100_500DM","sav500_1000DM","sav_unknown","emp_1year"    ,"emp_1_4years","emp_4_7years","emp_unemployed","inst_2","inst_3","inst_4","male:divorced","male:married","male:single","debtors_guarantor","debtors_none","res_2","res_3","res_4"  ,"prop_other","prop_realestate","proy_unknown","other_ins_none","other_inst_stores","hous_own","hous_rent","credits_2","credits_3","credits_4","job_employee","job_unemployed","job_unskilled"   ,"people_2","phone_yes","foreign_yes","duration_in_month","credit_amount","age","cicilan_per_bulan"]


In [ ]:
# Menggunakan scaling dengan standard scaler dari library sklearn
from sklearn.preprocessing import StandardScaler

# Memanggil fungsi standard scaler
sc = StandardScaler()

# Merubah nilai X dari Train dan Test menjadi standardisation
X_train_scale = sc.fit_transform(X_train)
X_test_scale = sc.fit_transform(X_test)
X_test_holdup_scale = sc.fit_transform(X_test_holdup)

In [ ]:
print(X_train_scale)

In [ ]:
## III.4. Modeling Decision Tree

# Menggunakan fungsi DecisionTree dari library sklearn
from sklearn.tree import DecisionTreeClassifier

# Fungsi Decision Tree, menggunakan opsi dasar saja
classifier = DecisionTreeClassifier()

# Melakukan model Decision Tree kepada data training
# Karena Decision Tree tidak memerlukan scaling, kita akan buat
# train dan test tanpa scaling
classifier.fit(X_train, y_train)

In [ ]:

import pydotplus
import graphviz

from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display, Image
from sklearn.externals.six import StringIO

# Untuk dapat menggunakan graphviz memerlukan instalasi library yang tepat, silahkan refer ke website berikut untuk petunjuk instalasi:
# https://stackoverflow.com/questions/27666846/pydot-invocationexception-graphvizs-executables-not-found

dot_data = StringIO()
export_graphviz(classifier, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())


In [ ]:

# Fungsi Decision Tree, menggunakan opsi max_depth
classifier = DecisionTreeClassifier(max_depth=3)

# Melakukan model Decision Tree kepada data training
classifier.fit(X_train, y_train)

dot_data = StringIO()
export_graphviz(classifier, out_file=dot_data, filled=True,rounded=True,
                special_characters=True, feature_names = kolom_short)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('decision_tree_2.png')
Image(graph.create_png())


In [ ]:
## III.5.1 Implementasi model dari data Train ke data Test 


# Memprediksi data test dengan menggunakan model Decision Tree
predicted = classifier.predict(X_test)
predicted_proba = classifier.predict_proba(X_test)

# Melihat hasil dari predict ke data test
print(predicted)

# Melihat hasil dari predict probabilitas ke data test
print(predicted_proba)

In [ ]:
## III.5.2 Evaluasi dengan Confusion Matrix 

# Menggunakan fungsi Confusion Matrix dari sklearn
from sklearn.metrics import confusion_matrix

# Kita akan cek hasil confusion matrix antara hasil sebenarnya dengan prediksi dari model ke data test
matrix = confusion_matrix(y_test, predicted)

# cek hasil matrix
print(matrix)

In [ ]:
## III.5.3 Classification Report

# Menggunakan fungsi Classification Report dari sklearn
from sklearn.metrics import classification_report

# Kita akan cek hasil confusion matrix antara hasil sebenarnya dengan prediksi dari model ke data test
report = classification_report(y_test, predicted)

# cek hasil matrix
print(report)

In [ ]:
# install scikitplot

!pip install -q scikit-plot

In [ ]:
## III.5.4 Gain and Lift chart 

import matplotlib.pyplot as plt
# Harap install library scikit-plot terlebih dahulu
import scikitplot as skplt

# Gain chart
skplt.metrics.plot_cumulative_gain(y_test, predicted_proba)
plt.show()

In [ ]:
# Lift chart
skplt.metrics.plot_lift_curve(y_test, predicted_proba)
plt.show()

In [ ]:
# K-S chart
skplt.metrics.plot_ks_statistic(y_test, predicted_proba)
plt.show()

In [ ]:
# Membuat chart ROC dan menghitung AUC
skplt.metrics.plot_roc(y_test, predicted_proba)

plt.show()

# Metode dengan Logistic Regression

In [ ]:
# Model Logistic Regression

from sklearn.linear_model import LogisticRegression

# Pembuatan model logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Hasil dari model: parameter koefisien
parameters = log_reg.coef_
print(parameters)

# Metode dengan Logistic Regression dengan Stats

In [ ]:
import numpy as np
import statsmodels.api as sm

model = sm.Logit(y_train, X_train)
result = model.fit(method='newton')

result.summary()

# Metode dengan Library Scorecardpy

In [ ]:
# install scorecardpy

!pip install -q scorecardpy

In [ ]:
# Credit Scoring using logistic regression with library scorecardpy
import scorecardpy as scpy

# scorecardpy library sudah memiliki fungsi untuk data split, scale, dan lainnya, oleh karena itu kita akan load df_prepare saja
dat = df_prepare

In [ ]:
dat.info()

In [ ]:
# Filter variabel secara otomatis
# Filter otomatis variabel dari fungsi scorecard, dengan tidak mengikutsertakan variabel yang terlalu banyak missing value dan nilai minimum Information Value
dt_s = scpy.var_filter(dat, y = 'default')
dt_s.drop(['installment_as_income_perc'], axis = 1, inplace = True)

In [ ]:
dt_s.info()

In [ ]:
# breaking dt_s menjadi training dan test dataset
train, test = scpy.split_df(dt_s, y='default').values()

# Melakukan WOE binning
bins = scpy.woebin(dt_s, y = 'default')

In [ ]:
# Melihat plot hasil dari WoE dan IV
scpy.woebin_plot(bins)

In [ ]:
# Hasil dari WoE dan IV secara kalkulasi
print(bins)

In [ ]:
# Membuat model scorecard

# converting train and test into woe values
train_woe = scpy.woebin_ply(train, bins)
test_woe = scpy.woebin_ply(test, bins)

y_train = train_woe.loc[:,'default']
X_train = train_woe.loc[:,train_woe.columns != 'default']
y_test = test_woe.loc[:,'default']
X_test = test_woe.loc[:,train_woe.columns != 'default']

# Membuat scorecard dengan menggunakan algoritma logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1', C=0.9, solver='saga', n_jobs=-1)
lr.fit(X_train, y_train)

In [ ]:
# predicted probability
# predicted proability
train_pred = lr.predict_proba(X_train)[:,1]
test_pred = lr.predict_proba(X_test)[:,1]

In [ ]:
# performance ks & roc ------
train_perf = scpy.perf_eva(y_train, train_pred, title = "train")
test_perf = scpy.perf_eva(y_test, test_pred, title = "test")

# score ------
card = scpy.scorecard(bins, lr, X_train.columns)
# credit score
train_score = scpy.scorecard_ply(train, card, print_step=0)
test_score = scpy.scorecard_ply(test, card, print_step=0)

# psi
scpy.perf_psi(
  score = {'train':train_score, 'test':test_score},
  label = {'train':y_train, 'test':y_test}
)